<div Style="text-align: center;line-height: 30px;font-size:32px;font-weight: bold"> Herramienta para el reescalado de imágenes en bloque</div>
***

In [1]:
%matplotlib inline
from __future__ import print_function
from ipywidgets import interact_manual, fixed

import os.path

from math import ceil

import numpy as np
import random

from skimage import io
from skimage.transform import rotate, rescale, resize

import matplotlib.pyplot as plt

import json

In [2]:
def rescale_coords(coords, factor_h, factor_v, h_limits, v_limits):
    rescale_cs = []
    for coord in coords:
        if(coord[0] > h_limits[0] and coord[2] < h_limits[1] and
           coord[1] > v_limits[0] and coord[3] < h_limits[1]):
            r_coord = []
            r_coord.append(coord[0] * factor_h)
            r_coord.append(coord[1] * factor_v)
            r_coord.append(coord[2] * factor_h)
            r_coord.append(coord[3] * factor_v)
            rescale_cs.append(r_coord)
    return rescale_cs

In [3]:
# Contamos el número de imágenes que tenemos en
# el directorio de las imágenes etiquetadas
import os, os.path
import re

path="../../rsc/img/picsCompressed/pics"
dest_path = "../../rsc/img/picsCompressed/reescaled/"

pattern = re.compile("^.*\.jpg$", re.IGNORECASE)

def list_images(path='../../rsc/img/Default'):
    images_list = []
    for name in os.listdir(path):
        json_name = name.split(".")[0] + ".json"
        if pattern.match(name) \
        and os.path.exists(path + "/" + json_name):
            images_list.append(path + "/" + name)
    return images_list

In [4]:
def read_coords_conversion(coords_dict):
    for k, coords in coords_dict.items():
        coords_mod = np.array(coords)
        coords_mod = coords_mod[:,[2,0,3,1]]
        coords_mod = coords_mod.tolist()
        coords_dict[k] = coords_mod
    return coords_dict

In [5]:
def save_coords_conversion(coords_dict):
    for k, coords in coords_dict.items():
        coords_mod = np.array(coords)
        coords_mod = coords_mod[:,[1,3,0,2]]
        coords_mod = coords_mod.tolist()
        coords_dict[k] = coords_mod
    return coords_dict

In [6]:
import copy

images_names_list = list_images(path)

if len(images_names_list) == 0:
    raise ValueError("Number of images must be greater than 0")

for image_path in images_names_list:
    
    original_image = io.imread(image_path)

    json_path = "../.." + image_path.split(".")[-2] + ".json"
    image_name = os.path.split(image_path)[1].split(".")[0]
    image_with_format = image_name + ".jpg"

    # Cargamos coordenadas, si existen,
    # y si no existe fichero de coordenadas
    # pasamos a la siguiente imagen
    if os.path.exists(json_path):
        with open(json_path) as jsonfile:
            coords_dict = json.load(jsonfile)
            coords_dict = coords_dict[image_with_format]
            coords_dict = read_coords_conversion(coords_dict)
    else:
        continue

    # Cambios de tamaño en la imagen
    mod_image = original_image[0:1920,0:1920]
    mod_image = resize(mod_image,(480,480),mode='constant')
    
    mod_coords = dict()
    for k, coords in coords_dict.items():
            mod_coords[k] = rescale_coords(coords, 
                                           480/1920, 480/1920, (0,1920), (0,1920)) 
    
    # Guardamos imagen
    io.imsave(dest_path + image_name + ".jpg", mod_image, quality=100)

    with open(dest_path + image_name + ".json", "w") as jsonfile2:
        mod_coords_dict = dict()
        mod_coords_dict[image_name +".jpg"] = save_coords_conversion(mod_coords)
        json.dump(mod_coords_dict, jsonfile2)

D:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
D:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
D:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
D:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
D:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
D:\Program Files\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: Use

IndexError: too many indices for array